In [1]:
#BAGIAN POSTPROCESSING

# Import necessary libraries
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
#from sklearn.preprocessing import LabelEncoder
#from sklearn.preprocessing import MinMaxScaler
import pickle as pk
import random as r

In [2]:
model = pk.load(open('recentmodel.sav', 'rb'))
data = pd.read_csv('Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv',encoding='cp1252')

D:\Skripsi Yusuf\python\Lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
D:\Skripsi Yusuf\python\Lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
import sklearn

print(sklearn.__version__)

1.2.2


In [4]:
testdata = data.drop(['Flow ID', ' Source IP', ' Timestamp', ' Destination IP', ' Label'], axis=1)
testdata = testdata.reset_index()

data2 = testdata.replace([np.inf, -np.inf], np.nan)
data2 = data2.dropna()
data2.head(10)

,index,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0,443,54865,6,3,2,0,12,0,6,...,1,20,0.0,0.0,0,0,0.0,0.0,0,0
1,1,80,55054,6,109,1,1,6,6,6,...,0,20,0.0,0.0,0,0,0.0,0.0,0,0
2,2,80,55055,6,52,1,1,6,6,6,...,0,20,0.0,0.0,0,0,0.0,0.0,0,0
3,3,443,46236,6,34,1,1,6,6,6,...,0,20,0.0,0.0,0,0,0.0,0.0,0,0
4,4,443,54863,6,3,2,0,12,0,6,...,1,20,0.0,0.0,0,0,0.0,0.0,0,0
5,5,443,54871,6,1022,2,0,12,0,6,...,1,20,0.0,0.0,0,0,0.0,0.0,0,0
6,6,443,54925,6,4,2,0,12,0,6,...,1,20,0.0,0.0,0,0,0.0,0.0,0,0
7,7,443,54925,6,42,1,1,6,6,6,...,0,20,0.0,0.0,0,0,0.0,0.0,0,0
8,8,443,9282,6,4,2,0,12,0,6,...,1,20,0.0,0.0,0,0,0.0,0.0,0,0
9,9,443,55153,6,4,2,0,37,0,31,...,1,20,0.0,0.0,0,0,0.0,0.0,0,0


In [5]:
prediction = model.predict(data2)

In [6]:
prediiiii = pd.DataFrame(prediction, columns=[' Label']).to_csv('prediction.csv')

In [7]:
data_pred = pd.read_csv('prediction.csv')

In [8]:
data_pred

,Unnamed: 0,Label
0,0,BENIGN
1,1,BENIGN
2,2,BENIGN
3,3,BENIGN
4,4,BENIGN
...,...,...
225706,225706,BENIGN
225707,225707,BENIGN
225708,225708,BENIGN
225709,225709,BENIGN


In [10]:
decoder = pd.read_csv('protocol_decode.csv')
decoder

,protocol,number
0,tcp,0
1,udp,1
2,pop,2
3,smtp,3
4,http,4
5,https,5
6,ftp,6
7,gopher,7
8,arp,8
9,dhcp,9


In [13]:
#Pastikan telah membuat predictedrules.txt dulu sebelum menjalankan kode

for index, attacks in enumerate(data_pred[' Label']):
  protocol = decoder.loc[decoder['number'] == data[' Protocol'][index]]['protocol'].values[0]
  strrules = f'alert {protocol} $EXTERNAL_NET any -> $HOME_NET any (msg:"{protocol} attack detected! -YYW";sid:{r.choice(range(1000000,2000000))};)\n'
  rulescompare = f'alert {protocol} $EXTERNAL_NET any -> $HOME_NET any (msg:"{protocol} attack detected! -YYW";'

  if attacks != 'BENIGN':
    with open("myrules.rules", "r") as rules:
      if rulescompare in rules.read():
        rules.close()
      else :
        rules.close()
        with open("myrules.rules", "a") as rules:
          rules.write(strrules)

rules.close()

In [12]:
data

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.5-104.16.207.165-54865-443-6,104.16.207.165,443,192.168.10.5,54865,6,7/7/2017 3:30,3,2,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,192.168.10.5-104.16.28.216-55054-80-6,104.16.28.216,80,192.168.10.5,55054,6,7/7/2017 3:30,109,1,1,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,192.168.10.5-104.16.28.216-55055-80-6,104.16.28.216,80,192.168.10.5,55055,6,7/7/2017 3:30,52,1,1,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,192.168.10.16-104.17.241.25-46236-443-6,104.17.241.25,443,192.168.10.16,46236,6,7/7/2017 3:30,34,1,1,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,192.168.10.5-104.19.196.102-54863-443-6,104.19.196.102,443,192.168.10.5,54863,6,7/7/2017 3:30,3,2,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225740,192.168.10.15-72.21.91.29-61374-80-6,72.21.91.29,80,192.168.10.15,61374,6,7/7/2017 5:02,61,1,1,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
225741,192.168.10.15-72.21.91.29-61378-80-6,72.21.91.29,80,192.168.10.15,61378,6,7/7/2017 5:02,72,1,1,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
225742,192.168.10.15-72.21.91.29-61375-80-6,72.21.91.29,80,192.168.10.15,61375,6,7/7/2017 5:02,75,1,1,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
225743,192.168.10.15-8.41.222.187-61323-80-6,8.41.222.187,80,192.168.10.15,61323,6,7/7/2017 5:02,48,2,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
